# City Locator

In [1]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("amaralibey/gsv-cities")

print("Path to dataset files:", path)

/home/go56pic/miniconda3/envs/tf-linux/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Path to dataset files: /home/go56pic/.cache/kagglehub/datasets/amaralibey/gsv-cities/versions/1


## City Locator (BIG Model)

### Import TensorFlow

In [2]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras import mixed_precision
from matplotlib import pyplot as plt

# Enable mixed precision for speed and lower memory when supported
mixed_precision.set_global_policy("mixed_float16")

2026-01-15 19:17:31.176515: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


### Load Image Dataset

In [ ]:
import os
from pathlib import Path
from collections import Counter

DATA_ROOT = Path(path)

# Recursively search for a directory that contains many class subfolders with files
def find_class_dir(root: Path, max_depth: int = 3, min_classes: int = 3):
    queue = [(root, 0)]
    best = None
    while queue:
        current, depth = queue.pop(0)
        if depth > max_depth:
            continue
        subdirs = [d for d in current.iterdir() if d.is_dir()]
        if len(subdirs) >= min_classes:
            has_files = any(any(f.is_file() for f in d.iterdir()) for d in subdirs)
            if has_files:
                return current, subdirs
            best = best or (current, subdirs)
        for sd in subdirs:
            queue.append((sd, depth + 1))
    return best if best else (root, [d for d in root.iterdir() if d.is_dir()])

DATA_DIR, class_dirs = find_class_dir(DATA_ROOT)
print(f"Using data directory: {DATA_DIR}")
if class_dirs:
    preview = [d.name for d in class_dirs][:10]
    suffix = "..." if len(class_dirs) > 10 else ""
    print(f"Found {len(class_dirs)} class folders: {preview}{suffix}")
else:
    print("Warning: No class subfolders found; please verify dataset structure.")

IMG_SIZE = 224  # smaller for faster MobileNetV2 training
BATCH_SIZE = 128  # lower if you hit OOM; raise if GPU allows
AUTOTUNE = tf.data.AUTOTUNE

train_ds = keras.utils.image_dataset_from_directory(
    DATA_DIR,
    labels="inferred",
    label_mode="categorical",
    image_size=(IMG_SIZE, IMG_SIZE),
    batch_size=BATCH_SIZE,
    shuffle=True,
    seed=42,
    validation_split=0.2,
    subset="training",
)

val_ds = keras.utils.image_dataset_from_directory(
    DATA_DIR,
    labels="inferred",
    label_mode="categorical",
    image_size=(IMG_SIZE, IMG_SIZE),
    batch_size=BATCH_SIZE,
    shuffle=True,
    seed=42,
    validation_split=0.2,
    subset="validation",
)

NUM_CLASSES = len(train_ds.class_names)
print(f"Detected NUM_CLASSES = {NUM_CLASSES}")

# Pipeline optimizations
train_ds = train_ds.prefetch(AUTOTUNE)
val_ds = val_ds.prefetch(AUTOTUNE)

Using data directory: /home/go56pic/.cache/kagglehub/datasets/amaralibey/gsv-cities/versions/1/Images
Found 23 class folders: ['Miami', 'Medellin', 'London', 'Brussels', 'Barcelona', 'Lisbon', 'Melbourne', 'Bangkok', 'Rome', 'Chicago']...
Found 529506 files belonging to 23 classes.
Using 423605 files for training.


I0000 00:00:1768501058.288554   31885 gpu_device.cc:2020] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 1481 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 3060, pci bus id: 0000:08:00.0, compute capability: 8.6


Found 529506 files belonging to 23 classes.
Using 105901 files for validation.
Detected NUM_CLASSES = 23


### Inspect Dataset Structure

In [4]:
from collections import defaultdict

# Summarize class counts (first 30 shown)
if class_dirs:
    counts = {}
    for d in class_dirs:
        counts[d.name] = sum(1 for f in d.rglob("*") if f.is_file())
    print(f"Total classes detected: {len(counts)}")
    top_names = list(counts.keys())[:30]
    for name in top_names:
        print(f"{name}: {counts[name]} images")
    if len(counts) > 30:
        print(f"... {len(counts) - 30} more classes not shown")
else:
    print("No classes detected; please inspect the dataset root manually.")

Total classes detected: 23
Miami: 43637 images
Medellin: 6024 images
London: 58672 images
Brussels: 14171 images
Barcelona: 15894 images
Lisbon: 27045 images
Melbourne: 28542 images
Bangkok: 22271 images
Rome: 24068 images
Chicago: 34091 images
Osaka: 22605 images
Minneapolis: 22326 images
Madrid: 14554 images
WashingtonDC: 11545 images
MexicoCity: 12801 images
Boston: 32616 images
BuenosAires: 8481 images
PRG: 17590 images
LosAngeles: 8891 images
PRS: 39963 images
Phoenix: 36251 images
OSL: 9756 images
TRT: 17712 images


### Add Noise / Randomness

In [5]:
# Data augmentation tuned for location invariance
data_augmentation = keras.Sequential(
    [
        layers.RandomFlip("horizontal"),
        layers.RandomRotation(0.12),
        layers.RandomZoom(0.15),
        layers.RandomTranslation(0.1, 0.1),
        layers.RandomContrast(0.2),
        layers.RandomBrightness(0.2),
    ],
    name="data_augmentation",
)

### Setup and Build Training Model

In [6]:
# Base model: MobileNetV2 for speed/size
base_model = keras.applications.MobileNetV2(
    include_top=False,
    weights="imagenet",
    input_shape=(IMG_SIZE, IMG_SIZE, 3),
    alpha=1.0,
)

# Stage 1: freeze backbone
base_model.trainable = False

inputs = keras.Input(shape=(IMG_SIZE, IMG_SIZE, 3))
x = data_augmentation(inputs)
x = keras.applications.mobilenet_v2.preprocess_input(x)

x = base_model(x, training=False)
x = layers.GlobalAveragePooling2D(name="global_avg_pool")(x)

# Lightweight classification head
x = layers.Dropout(0.2)(x)
x = layers.Dense(256, activation="relu", kernel_regularizer=keras.regularizers.l2(1e-4))(x)
x = layers.Dropout(0.2)(x)
outputs = layers.Dense(NUM_CLASSES, activation="softmax", dtype="float32", name="predictions")(x)

model = keras.Model(inputs, outputs, name="mobilenetv2_city_locator")

model.compile(
    optimizer=keras.optimizers.Adam(learning_rate=3e-4),
    loss=keras.losses.CategoricalCrossentropy(label_smoothing=0.05),
    metrics=[
        "accuracy",
        keras.metrics.TopKCategoricalAccuracy(k=5, name="top_5_acc"),
    ],
)

9406464/9406464 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


### Execute Training Model

In [ ]:
# Show the model structure and run a short training loop so the cell produces output
model.summary()
history = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=6,  # shorter for MobileNetV2
    verbose=1,
)

Model: "mobilenetv2_city_locator"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_1 (InputLayer)      │ (None, 224, 224, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ data_augmentation (Sequential)  │ (None, 224, 224, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ true_divide (TrueDivide)        │ (None, 224, 224, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ subtract (Subtract)             │ (None, 224, 224, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ mobilenetv2_1.00_224            │ (None, 7, 7, 1280)     │     2,257,984 │
│ (Functional)                    │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_avg_pool                 │ (None, 1280)           │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 1280)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 256)            │       327,936 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ predictions (Dense)             │ (None, 23)             │         5,911 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,591,831 (9.89 MB)

 Trainable params: 333,847 (1.27 MB)

 Non-trainable params: 2,257,984 (8.61 MB)

Epoch 1/6


2026-01-15 19:18:07.131192: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:473] Loaded cuDNN version 91701


 522/6619 ━━━━━━━━━━━━━━━━━━━━ 7:32 74ms/step - accuracy: 0.1617 - loss: 3.0250 - top_5_acc: 0.4540

### Plot Epochs

In [ ]:
plt.plot(history.epoch, history.history["loss"])
plt.title("Train Loss Curve")
plt.show()

plt.plot(history.epoch, history.history["accuracy"])
plt.title("Train Accuracy Curve")
plt.show()

### Fine-Tuning Model

In [ ]:
# Unfreeze top blocks of MobileNetV2 for fine-tuning
base_model.trainable = True

fine_tune_at = int(len(base_model.layers) * 0.8)

for i, layer in enumerate(base_model.layers):
    layer.trainable = i >= fine_tune_at
    if isinstance(layer, layers.BatchNormalization):
        layer.trainable = False

model.compile(
    optimizer=keras.optimizers.Adam(learning_rate=1e-4),
    loss=keras.losses.CategoricalCrossentropy(label_smoothing=0.05),
    metrics=[
        "accuracy",
        keras.metrics.TopKCategoricalAccuracy(k=5, name="top_5_acc"),
    ],
)

# Train again with early stopping
early_stopping = keras.callbacks.EarlyStopping(
    monitor="val_accuracy",
    patience=2,
    restore_best_weights=True,
)

history_finetune = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=5,
    callbacks=[early_stopping],
    verbose=1,
)